In [4]:
import AdventOfCode

input <- map readInt . words <$> dayString 6

In [20]:
import qualified Data.Vector.Unboxed as V
import qualified Data.Vector.Unboxed.Mutable as M
import Data.List
import Data.Maybe
import Control.Monad.ST
import Data.STRef

maxIndex l = fromJust $ elemIndex (maximum l) l

go :: M.STVector s Int -> STRef s Int -> STRef s Int -> ST s (M.STVector s Int)
go vector index blocks = do
    count <- readSTRef blocks
    i <- readSTRef index
    let i' = i `mod` M.length vector
    if count == 0
        then return vector
        else do
            M.modify vector (+1) i'
            writeSTRef index (i'+1)
            writeSTRef blocks (count-1)
            go vector index blocks

In [72]:
empty :: M.STVector s Int -> Int -> ST s Int
empty mut i = do
    blocks <- M.read mut i
    M.write mut i 0
    return blocks

redistribute :: [Int] -> [Int]
redistribute l = runST $ do
    mut <- V.thaw (V.fromList l)
    let i = maxIndex l
    blocks <- newSTRef =<< empty mut i
    pointer <- newSTRef (i+1)
    go mut pointer blocks
    frozen <- V.freeze mut
    return $ V.toList frozen

loop = iterate redistribute input

part1 moves i = let
    (before, after) = splitAt i moves
    current = head after
    in if current `elem` before then i else part1 moves (i+1)

part1 loop 0

12841

In [70]:
part2 moves i = let
    (before, after) = splitAt i moves
    current = head after
    in if current `elem` before then i - fromJust (elemIndex current before) else part2 moves (i+1)

part2 loop 0

8038